In [1]:
import os
# set the current working directory to the deployed package folder. This is required by isaac.
# This cell should only run once.
os.chdir("../../../")
os.getcwd()

'/home/robolab/.cache/bazel/_bazel_robolab/ab0f245373231211c3f02101438285e2/execroot/com_nvidia_isaac_sdk/bazel-out/k8-opt/bin/packages/universal_robots/digital_twin_app/simple_joint_control.runfiles/com_nvidia_isaac_sdk'

In [2]:
from IPython.display import display
import json
import numpy as np
import time

from packages.pyalice import Application, Codelet, Message
from packages.pyalice.gui.composite_widget import CompositeWidget

np.set_printoptions(precision=3)

/home/robolab/.local/lib/python3.6/site-packages/numba/core/errors.py:154: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)


In [3]:
# A Python codelet for joint control through widget
class JointPositionControl(Codelet):
    def start(self):
        self.rx = self.isaac_proto_rx("CompositeProto", "state")
        self.tx = self.isaac_proto_tx("CompositeProto", "command")
        
        joints = self.config.joints
        limits = self.config.limits
        measure = self.config.measure
        self.stop_control = 3
        self._widget = CompositeWidget(joints, measure, limits)
        if self._widget is None:
            report_failure("Cannot create valid widget")
            return
        display(self._widget.panel)

        self.tick_periodically(0.1)

    def tick(self):
        state_msg = self.rx.message
        if state_msg is None:
            return
        self._widget.composite = state_msg

        self.tx._msg = self._widget.composite
        if self.tx._msg is not None:
            self.tx.publish()
            
# set kinematic file and get list of joints
kinematic_file = "apps/assets/kinematic_trees/ur10.kinematic.json"
joints = []
with open(kinematic_file,'r') as fd:
    kt = json.load(fd)
    for link in kt['links']:
        if 'motor' in link and link['motor']['type'] != 'constant':
            joints.append(link['name'])
print(joints)

# Obtain the IP from UR Console and update this
IP = "192.168.0.100"
# Robot generation that you are going to control "e-series" or "cb3"
GENERATION = "cb3"

['shoulder_pan_joint', 'shoulder_lift_joint', 'elbow_joint', 'wrist_1_joint', 'wrist_2_joint', 'wrist_3_joint']


## Merge both into one app

UR10 Hardware Joint Control
======

In [4]:
app_hw = Application(name="simple_joint_control_ur_hardware", modules=["sight"])

if GENERATION == "e-series":    
    app_hw.load(filename="packages/universal_robots/ur_robot_driver/apps/ur_eseries_robot.subgraph.json", prefix="ur")
elif GENERATION == "cb3":
    app_hw.load(filename="packages/universal_robots/ur_robot_driver/apps/ur_cb3_robot.subgraph.json", prefix="ur")
else:
    Exception("Unknown robot generation")

# Load components for configuration
ur_interface = app_hw.nodes["ur.subgraph"]["interface"]
ur_controller = app_hw.nodes["ur.controller"]["ScaledMultiJointController"]
ur_driver = app_hw.nodes["ur.universal_robots"]["UniversalRobots"]

# configs
ur_controller.config.control_mode = "joint speed"
ur_driver.config.control_mode = "joint speed"
ur_driver.config.robot_ip = IP
ur_driver.config.headless_mode = False

# add pycodelet for joint position control
widget_node = app_hw.add("command_generator")
joint_commander = widget_node.add(JointPositionControl)
joint_commander.config.joints = joints
joint_commander.config.limits = [[-2*np.pi, 2*np.pi]] * len(joints)
joint_commander.config.measure = 'position'

# edges
app_hw.connect(joint_commander, "command", ur_interface, "joint_target")
app_hw.connect(ur_interface, "arm_state", joint_commander, "state")

# Enable sight
widget = app_hw.add("sight").add(app_hw.registry.isaac.sight.SightWidget, "simple_joint_control")
widget.config.type = "plot"
widget.config.channels = [
  {
    "name": "ur.universal_robots/UniversalRobots/shoulder_pan_joint"
  },
  {
    "name": "ur.universal_robots/UniversalRobots/shoulder_lift_joint"
  },
  {
    "name": "ur.universal_robots/UniversalRobots/elbow_joint"
  },
  {
    "name": "ur.universal_robots/UniversalRobots/wrist_1_joint"
  },
  {
    "name": "ur.universal_robots/UniversalRobots/wrist_2_joint"
  },
  {
    "name": "ur.universal_robots/UniversalRobots/wrist_3_joint"
  }

]


2022-01-12 17:07:55,620 DEBUG Binding PyCodelet command_generator/PyCodelet


LQR planner Simulation part
-----

In [ ]:
app_sim = Application(name="simple_joint_control_lqr_ur10_sim")

# load subgraphs
app_sim.load(filename="packages/planner/apps/multi_joint_lqr_control.subgraph.json", prefix="lqr")
app_sim.load(filename="packages/navsim/apps/navsim_tcp.subgraph.json", prefix="simulation")
# edges
simulation_node = app_sim.nodes["simulation.interface"]
lqr_interface = app_sim.nodes["lqr.subgraph"]["interface"]
app_sim.connect(simulation_node["output"], "joint_state", lqr_interface, "joint_state")
app_sim.connect(lqr_interface, "joint_command", simulation_node["input"], "joint_position")
# configs
app_sim.nodes["lqr.kinematic_tree"]["KinematicTree"].config.kinematic_file = kinematic_file
lqr_planner = app_sim.nodes["lqr.local_plan"]["MultiJointLqrPlanner"]
lqr_planner.config.speed_min = [-1.0] * len(joints)
lqr_planner.config.speed_max = [1.0] * len(joints)
lqr_planner.config.acceleration_min = [-1.0] * len(joints)
lqr_planner.config.acceleration_max = [1.0] * len(joints)

# add pycodelet JointControl for arm control
joint_commander = app_sim.add("command_generator").add(JointPositionControl)
joint_commander.config.joints = joints
joint_commander.config.limits = [[-2*np.pi, 2*np.pi]] * len(joints)
joint_commander.config.measure = "position"

app_sim.connect(joint_commander, "command", lqr_interface, "joint_target")
app_sim.connect(simulation_node["output"], "joint_state", joint_commander, "state")


### start application

In [5]:
app_hw.start()

2022-01-12 17:07:58,810 DEBUG Launching isaac core
2022-01-12 17:07:58,812 DEBUG Launching pycodelet threads
2022-01-12 17:07:58,813 DEBUG Launching command_generator/PyCodelet


In [ ]:
app_sim.start()

### stop application

In [6]:
app_hw.stop()

2022-01-12 17:08:46,899 DEBUG Stopped command_generator/PyCodelet
2022-01-12 17:08:46,900 DEBUG Python Codelets All stopped...


In [ ]:
app_sim.stop()

## Later work

UR10 Digital IO Control
=====

In [ ]:
app = Application(name="io_control_ur_hardware", modules=["sight"])

if GENERATION == "e-series":    
    app.load(filename="packages/universal_robots/ur_robot_driver/apps/ur_eseries_robot.subgraph.json", prefix="ur")
elif GENERATION == "cb3":
    app.load(filename="packages/universal_robots/ur_robot_driver/apps/ur_cb3_robot.subgraph.json", prefix="ur")
else:
    Exception("Unknown robot generation")

# Load components for configuration
ur_interface = app.nodes["ur.subgraph"]["interface"]
ur_driver = app.nodes["ur.universal_robots"]["UniversalRobots"]

io_names = ur_driver.config.tool_digital_out_names
ur_driver.config.robot_ip = IP

# add pycodelet for digitalIO control
widget_node = app.add("command_generator")
io_commander = widget_node.add(JointPositionControl)
io_commander.config.joints = io_names
io_commander.config.limits = [[0, 1]] * len(io_names)
io_commander.config.measure = 'none'

# Edges
app.connect(io_commander, "command", ur_interface, "io_command")
app.connect(ur_interface, "io_state", io_commander, "state")

# Enable sight
widget = app.add("sight").add(app.registry.isaac.sight.SightWidget, "shuffle_box_hardware")
widget.config.type = "plot"
widget.config.channels = [
  {
    "name": "ur.universal_robots/UniversalRobots/tool_digital_out_0"
  },
  {
    "name": "ur.universal_robots/UniversalRobots/tool_digital_out_1"
  },
  {
    "name": "ur.universal_robots/UniversalRobots/tool_digital_in_0"
  },
  {
    "name": "ur.universal_robots/UniversalRobots/tool_digital_in_1"
  }
]

app.start()

In [ ]:
app.stop()